<a href="https://colab.research.google.com/github/FionaNalianya/DataEngineering-intro/blob/main/Introduction_to_Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AfterWork: Introduction to Data Engineering

# Example

In this practice notebook, you will learn how to extract data, transform it, and then load the transformed data to your desired destination. For the sake of simplicity, this notebook will focus on extracting from and loading to CSV files although it is worth keeping in mind that, in real-world scenarios, you might work with a variety of data sources and data files.

The link to the `sales_transaction.csv` file can be found [here](https://tinyurl.com/34c74cpd).

## The Extract Stage

In [2]:
# First import the necessary libraries
import pandas as pd
from datetime import datetime

In [3]:
# Extract the data from the `sales_transaction.csv` file and load it into a Pandas DataFrame
sales = pd.read_csv("https://tinyurl.com/34c74cpd")

# Preview the first 5 rows
sales.head()

,Transaction_ID,Customer_ID,Product_ID,Purchase_Date,Quantity,Price
0,1,101,P1,2023-01-05,'2',20.0
1,2,102,P2,2023-01-10,'3',15.0
2,3,103,P3,2023-01-15,NaN,5.0
3,4,101,P1,2023-01-05,'2',20.0
4,5,102,P1,2023-02-02,'1',20.0


In [4]:
# Check the number of rows and columns
sales.shape

(30, 6)

Our dataset has 30 rows and 6 columns.

In [5]:
# Check the datatypes
sales.dtypes

Transaction_ID      int64
Customer_ID         int64
Product_ID         object
Purchase_Date      object
Quantity           object
Price             float64
dtype: object

In [6]:
# Check missing values
sales.isna().sum()

Transaction_ID    0
Customer_ID       0
Product_ID        2
Purchase_Date     1
Quantity          2
Price             1
dtype: int64

## The Transform Stage

We will drop the `Transaction_ID` column since it is irrelevant.

In [7]:
sales.drop(columns=['Transaction_ID'], inplace=True)

We will convert the `Purchase_Date` column to datetime format.

In [8]:
sales["Purchase_Date"] = pd.to_datetime(sales["Purchase_Date"])

We will remove the quotes surrounding the values in `Quantity` and convert that column to numeric.

In [9]:
sales["Quantity"] = pd.to_numeric(sales["Quantity"].str.strip("'"))

While there are various ways to deal with missing values, we will drop them in this case.

In [10]:
sales.dropna(inplace=True)

We will drop duplicates if there are any.

In [11]:
sales.drop_duplicates(inplace=True)

Next, we will create a new column called `Total_Sales` which is obtained by multiplying `Quantity` with `Price`.

In [12]:
sales["Total_Sales"] = sales["Quantity"] * sales["Price"]

In [13]:
# Preview the final dataset
sales.head()

,Customer_ID,Product_ID,Purchase_Date,Quantity,Price,Total_Sales
0,101,P1,2023-01-05,2.0,20.0,40.0
1,102,P2,2023-01-10,3.0,15.0,45.0
4,102,P1,2023-02-02,1.0,20.0,20.0
6,104,P3,2023-02-20,2.0,5.0,10.0
7,101,P4,2023-03-01,1.0,10.0,10.0


In [14]:
# Check shape
sales.shape

(21, 6)

In [15]:
# Ensure there are no missing values
sales.isna().sum()

Customer_ID      0
Product_ID       0
Purchase_Date    0
Quantity         0
Price            0
Total_Sales      0
dtype: int64

In [16]:
# Aggregate sales data by product
sales_by_product = sales.groupby("Product_ID").agg(
    Total_Quantity=("Quantity", "sum"),
    Total_Revenue=("Total_Sales", "sum")
)
print("Sales by product:")
sales_by_product

Sales by product:


,Total_Quantity,Total_Revenue
Product_ID,,
P1,13.0,260.0
P2,15.0,225.0
P3,12.0,60.0
P4,6.0,60.0


In [17]:
# Aggregate sales data by customer
sales_by_customer = sales.groupby("Customer_ID").agg(
    Total_Quantity=("Quantity", "sum"),
    Total_Revenue=("Total_Sales", "sum")
)
print("Sales by customer:")
sales_by_customer

Sales by customer:


,Total_Quantity,Total_Revenue
Customer_ID,,
101,6.0,105.0
102,8.0,85.0
103,1.0,10.0
104,4.0,40.0
105,6.0,60.0
106,7.0,130.0
107,2.0,40.0
108,3.0,45.0
109,3.0,15.0


## The Load Stage

In [18]:
# Save to a new CSV file
sales.to_csv("clean_sales_transactions.csv", index=False)
sales_by_product.to_csv("sales_by_product.csv", header=True)
sales_by_customer.to_csv("sales_by_customer.csv", header=True)

# Challenge 1

Carry out ETL on the [movie_ratings.csv](https://tinyurl.com/2kd3e97z) dataset provided. Some actions you could carry out in the Transform stage include:
- Calculating the average rating for each movie.
- Calculating the number of ratings per user.
- Grouping movies into rating categories (e.g., high-rated, low-rated).

In [20]:
# Extract

import pandas as pd

# Load the dataset
ratings = pd.read_csv("https://tinyurl.com/2kd3e97z")

# Check shape, datatypes, and missing values
print("Shape of the dataset:", ratings.shape)
print("\nData types:\n", ratings.dtypes)
print("\nNumber of missing values:\n", ratings.isnull().sum())

Shape of the dataset: (21, 4)

Data types:
 User_ID        int64
Movie_ID     float64
Rating       float64
Timestamp     object
dtype: object

Number of missing values:
 User_ID      0
Movie_ID     1
Rating       2
Timestamp    0
dtype: int64


In [21]:
# Transform

# Remove missing values and duplicates
ratings = ratings.dropna().drop_duplicates()

# Calculate average rating for each movie
average_ratings = ratings.groupby("Movie_ID")["Rating"].mean()

# Calculate number of ratings per user
ratings_per_user = ratings.groupby("User_ID")["Rating"].count()

# Define rating categories
def categorize_rating(rating):
    if rating >= 4:
        return "High-Rated"
    else:
        return "Low-Rated"

# Group movies into rating categories
ratings["Rating_Category"] = ratings["Rating"].apply(categorize_rating)

In [22]:
# Load

# Save transformed dataset and grouping into CSV files
ratings.to_csv("transformed_movie_ratings.csv", index=False)
average_ratings.to_csv("average_ratings_per_movie.csv", header=True)
ratings_per_user.to_csv("ratings_per_user.csv", header=True)